In [1]:
using Pkg
Pkg.activate()
Pkg.instantiate()
import Pkg
using MixedModels
using RCall
using DataFrames, Tables
using Random
using CSV
using RData
using CategoricalArrays
using Statistics

 Activating environment at `C:\Users\bjoer\.julia\environments\v1.4\Project.toml`



R version 3.6.1 (2019-07-05) -- "Action of the Toes"
Copyright (C) 2019 The R Foundation for Statistical Computing
Platform: x86_64-w64-mingw32/x64 (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.



In [38]:
R"""

###Pull the whole repository and paste the path to the the local GitHub repository here:
require(dplyr)
require(tidyverse)
require(lme4)
set.seed(121)
require(lmerTest)

binomial_smooth <- function(...) {
  geom_smooth(method = "glm", method.args = list(family = "binomial"), ...)}

setwd("C:/Users/bjoer/Documents/GitHub/Gravity-and-Self-Motion/")

source("Utilities/parabolic.r")

SimulateDataframe = function(n, SelfMotion, Gravity, IntervalVerticalVelocity, IntervalHorizontalVelocity, 
                             reps, ObjectDirection,Orientation, SlantInterval){
 
 SlantInterval = DegreeToRadians(SlantInterval)
 
 ID = paste0("s",1:n)
 reps = 1:reps
    
 Dataframe = expand.grid(ID = ID, 
                         SelfMotion = SelfMotion,
                         Gravity = Gravity,
                         reps = reps,
                         ObjectDirection = ObjectDirection,
                         Orientation = Orientation)

 Dataframe = Dataframe %>%
   filter(!(Orientation == 1 & Gravity == -9.81))

 Dataframe = Dataframe %>%
   mutate(VerticalVelocity = runif(length(ID),IntervalVerticalVelocity[1],IntervalVerticalVelocity[2]),
          HorizontalVelocity = runif(length(ID),IntervalHorizontalVelocity[1],IntervalHorizontalVelocity[2])*ObjectDirection,
          Percentage = runif(length(ID),0.575,0.725),
          Slant = runif(length(ID),SlantInterval[1],SlantInterval[2]),
          y1 = (-((VerticalVelocity/(HorizontalVelocity*tan(Slant)))-1) + (((VerticalVelocity/(HorizontalVelocity*tan(Slant)))-1)^2)^0.5)/
            ((Gravity/((HorizontalVelocity^2)*((tan(Slant))^2)))*(-2)),
          y2 = (-((VerticalVelocity/(HorizontalVelocity*tan(Slant)))-1) - (((VerticalVelocity/(HorizontalVelocity*tan(Slant)))-1)^2)^0.5)/
            ((Gravity/((HorizontalVelocity^2)*((tan(Slant))^2)))*(-2)),
          y = case_when(
            y1 != 0 ~ y1,
            y2 != 0 ~ y2),
          t1 = (-VerticalVelocity+(VerticalVelocity^2-4*Gravity*y)^0.5)/Gravity,
          t2 = (-VerticalVelocity-(VerticalVelocity^2-4*Gravity*y)^0.5)/Gravity,
          FlightDuration = case_when(
            abs(t1) > abs(t2) ~ t1, 
            abs(t2) > abs(t1) ~ t2,
          ),
          Distance = HorizontalVelocity*FlightDuration)

  Dataframe = Dataframe %>%
    mutate(#FlightDuration = abs(getTimetoYIsZero(VerticalVelocity,Gravity)),
           #Distance = abs(HorizontalVelocity*FlightDuration),
           VisibleTime = FlightDuration*Percentage,
           OccludedTime = abs(FlightDuration)-abs(VisibleTime),
           VisibleDistance = VisibleTime*HorizontalVelocity,
           OccludedDistance = abs(Distance)-abs(VisibleDistance))

  Dataframe = Dataframe %>%
    mutate(SD_Factor_Time = 
             case_when(#baseline: 1 = 100%; 
               #Main Effect - Gravity: +0.2 if -1g or sidewards;
               #
               #Main Effect - Selfmotion: +0.2 if parallel motion (reduced by 0.1 if 1g)
               #                          +0.1 if orthogonal motion (reduced by 0.05 if 1g)
               #
               #Interaction: -0.05 if parallel motion and 1g
               #             -0.025 if orthogonal motion and 1g
               Orientation == 0 & SelfMotion == 1 & Gravity ==  9.81 ~ 1 + 0.0 + 0.20 - 0.10,
               Orientation == 0 & SelfMotion == 1 & Gravity == -9.81 ~ 1 + 0.2 + 0.20 - 0.00,
               Orientation == 0 & SelfMotion == 2 & Gravity ==  9.81 ~ 1 + 0.0 + 0.10 - 0.05,
               Orientation == 0 & SelfMotion == 2 & Gravity == -9.81 ~ 1 + 0.2 + 0.10 - 0.00,
               Orientation == 0 & SelfMotion == 3 & Gravity ==  9.81 ~ 1 + 0.0 + 0.20 - 0.10,
               Orientation == 0 & SelfMotion == 3 & Gravity == -9.81 ~ 1 + 0.2 + 0.20 - 0.00,
               Orientation == 0 & SelfMotion == 4 & Gravity ==  9.81 ~ 1 + 0.0 + 0.10 - 0.05,
               Orientation == 0 & SelfMotion == 4 & Gravity == -9.81 ~ 1 + 0.2 + 0.10 - 0.00,
               Orientation == 0 & SelfMotion == 5 & Gravity ==  9.81 ~ 1 + 0.0 + 0.00 - 0.00,
               Orientation == 0 & SelfMotion == 5 & Gravity == -9.81 ~ 1 + 0.2 + 0.00 - 0.00,
               Orientation == 1 & SelfMotion == 1 & Gravity ==  9.81 ~ 1 + 0.2 + 0.10 - 0.00,
               Orientation == 1 & SelfMotion == 2 & Gravity ==  9.81 ~ 1 + 0.2 + 0.20 - 0.00,
               Orientation == 1 & SelfMotion == 3 & Gravity ==  9.81 ~ 1 + 0.2 + 0.10 - 0.00,
               Orientation == 1 & SelfMotion == 4 & Gravity ==  9.81 ~ 1 + 0.2 + 0.20 - 0.00,
               Orientation == 1 & SelfMotion == 5 & Gravity ==  9.81 ~ 1 + 0.2 + 0.00 - 0.00),
          Mean_Factor_Time =
             case_when(#baseline 1.00 for 1g and static
               #Main Effect - Gravity: +0.1 if -1g/+0.075 if sidewards
               #
               #Main Effect - Selfmotion: -0.08 if right and sidewards
               #                          +0.08 if left and sidewards
               #                          +0.08 if up and -1g
               #                          -0.08 if down and -1g
               #
               #Interaction: -0.04 if up and 1g
               #             +0.04 if down and 1g
               Orientation == 0 & SelfMotion == 1 & Gravity ==  9.81 ~ 1 + 0.000 - 0.08 + 0.04,
               Orientation == 0 & SelfMotion == 1 & Gravity == -9.81 ~ 1 + 0.100 + 0.08 + 0.00,
               Orientation == 0 & SelfMotion == 2 & Gravity ==  9.81 ~ 1 + 0.000 + 0.00 + 0.00,
               Orientation == 0 & SelfMotion == 2 & Gravity == -9.81 ~ 1 + 0.100 + 0.00 + 0.00,
               Orientation == 0 & SelfMotion == 3 & Gravity ==  9.81 ~ 1 + 0.000 + 0.08 - 0.04,
               Orientation == 0 & SelfMotion == 3 & Gravity == -9.81 ~ 1 + 0.100 - 0.08 + 0.00,
               Orientation == 0 & SelfMotion == 4 & Gravity ==  9.81 ~ 1 + 0.000 + 0.00 + 0.00,
               Orientation == 0 & SelfMotion == 4 & Gravity == -9.81 ~ 1 + 0.100 + 0.00 + 0.00,
               Orientation == 0 & SelfMotion == 5 & Gravity ==  9.81 ~ 1 + 0.000 + 0.00 + 0.00,
               Orientation == 0 & SelfMotion == 5 & Gravity == -9.81 ~ 1 + 0.100 + 0.00 + 0.00,
               Orientation == 1 & SelfMotion == 1 & Gravity ==  9.81 ~ 1 + 0.075 + 0.00 + 0.00,
               Orientation == 1 & SelfMotion == 2 & Gravity ==  9.81 ~ 1 + 0.075 - 0.08 + 0.00,
               Orientation == 1 & SelfMotion == 3 & Gravity ==  9.81 ~ 1 + 0.075 + 0.00 + 0.00,
               Orientation == 1 & SelfMotion == 4 & Gravity ==  9.81 ~ 1 + 0.075 + 0.08 + 0.00,
               Orientation == 1 & SelfMotion == 5 & Gravity ==  9.81 ~ 1 + 0.075 + 0.00 + 0.00)
             )

  Dataframe = Dataframe %>%
    mutate(SD_Factor_Space = 
             case_when(
               #+0.05 when -1g
               #+0.05 when sidewards
               #+0.1 when left or right when selfmotion
               #-0.05 if 1g and selfmotion
               
               Orientation == 0 & SelfMotion == 1 & Gravity ==  9.81 ~ 1 + 0.0 + 0.2 - 0.1,
               Orientation == 0 & SelfMotion == 1 & Gravity == -9.81 ~ 1 + 0.2 + 0.2 - 0.0,
               Orientation == 0 & SelfMotion == 2 & Gravity ==  9.81 ~ 1 + 0.0 + 0.2 - 0.1,
               Orientation == 0 & SelfMotion == 2 & Gravity == -9.81 ~ 1 + 0.2 + 0.2 - 0.0,
               Orientation == 0 & SelfMotion == 3 & Gravity ==  9.81 ~ 1 + 0.0 + 0.2 - 0.1,
               Orientation == 0 & SelfMotion == 3 & Gravity == -9.81 ~ 1 + 0.2 + 0.2 - 0.0,
               Orientation == 0 & SelfMotion == 4 & Gravity ==  9.81 ~ 1 + 0.0 + 0.2 - 0.1,
               Orientation == 0 & SelfMotion == 4 & Gravity == -9.81 ~ 1 + 0.2 + 0.2 - 0.0,
               Orientation == 0 & SelfMotion == 5 & Gravity ==  9.81 ~ 1 + 0.0 + 0.0 - 0.0,
               Orientation == 0 & SelfMotion == 5 & Gravity == -9.81 ~ 1 + 0.2 + 0.0 - 0.0,
               
               Orientation == 1 & SelfMotion == 1 & Gravity ==  9.81 ~ 1 + 0.2 + 0.2 - 0.00,
               Orientation == 1 & SelfMotion == 2 & Gravity ==  9.81 ~ 1 + 0.2 + 0.2 - 0.00,
               Orientation == 1 & SelfMotion == 3 & Gravity ==  9.81 ~ 1 + 0.2 + 0.2 - 0.00,
               Orientation == 1 & SelfMotion == 4 & Gravity ==  9.81 ~ 1 + 0.2 + 0.2 - 0.00,
               Orientation == 1 & SelfMotion == 5 & Gravity ==  9.81 ~ 1 + 0.2 + 0.0 - 0.00))

  Dataframe = Dataframe %>%
             mutate(Mean_Factor_Space =
             case_when(
               Orientation == 0 & HorizontalVelocity < 0 & SelfMotion == 1 & Gravity ==  9.81 ~ 1 + 0.00 + 0.10 - 0.05,
               Orientation == 0 & HorizontalVelocity < 0 & SelfMotion == 2 & Gravity ==  9.81 ~ 1 + 0.00 - 0.10 + 0.05,
               Orientation == 0 & HorizontalVelocity < 0 & SelfMotion == 3 & Gravity ==  9.81 ~ 1 + 0.00 - 0.10 + 0.05,
               Orientation == 0 & HorizontalVelocity < 0 & SelfMotion == 4 & Gravity ==  9.81 ~ 1 + 0.00 + 0.10 - 0.05,
               Orientation == 0 & HorizontalVelocity < 0 & SelfMotion == 5 & Gravity ==  9.81 ~ 1 + 0.00 + 0.00 + 0.00,

               Orientation == 0 & HorizontalVelocity < 0 & SelfMotion == 1 & Gravity == -9.81 ~ 1 - 0.05 - 0.10 + 0.00,
               Orientation == 0 & HorizontalVelocity < 0 & SelfMotion == 2 & Gravity == -9.81 ~ 1 - 0.05 - 0.10 + 0.00,
               Orientation == 0 & HorizontalVelocity < 0 & SelfMotion == 3 & Gravity == -9.81 ~ 1 - 0.05 + 0.10 + 0.00,
               Orientation == 0 & HorizontalVelocity < 0 & SelfMotion == 4 & Gravity == -9.81 ~ 1 - 0.05 + 0.10 + 0.00,
               Orientation == 0 & HorizontalVelocity < 0 & SelfMotion == 5 & Gravity == -9.81 ~ 1 - 0.05 + 0.00 + 0.00,
                              
               Orientation == 0 & HorizontalVelocity > 0 & SelfMotion == 1 & Gravity ==  9.81 ~ 1 + 0.00 - 0.10 + 0.05,
               Orientation == 0 & HorizontalVelocity > 0 & SelfMotion == 2 & Gravity ==  9.81 ~ 1 + 0.00 - 0.10 + 0.05,
               Orientation == 0 & HorizontalVelocity > 0 & SelfMotion == 3 & Gravity ==  9.81 ~ 1 + 0.00 + 0.10 - 0.05,
               Orientation == 0 & HorizontalVelocity > 0 & SelfMotion == 4 & Gravity ==  9.81 ~ 1 + 0.00 + 0.10 - 0.05,
               Orientation == 0 & HorizontalVelocity > 0 & SelfMotion == 5 & Gravity ==  9.81 ~ 1 + 0.00 + 0.00 + 0.00,

               Orientation == 0 & HorizontalVelocity > 0 & SelfMotion == 1 & Gravity == -9.81 ~ 1 + 0.05 + 0.10 + 0.00,
               Orientation == 0 & HorizontalVelocity > 0 & SelfMotion == 2 & Gravity == -9.81 ~ 1 + 0.05 - 0.10 + 0.00,               
               Orientation == 0 & HorizontalVelocity > 0 & SelfMotion == 3 & Gravity == -9.81 ~ 1 + 0.05 - 0.10 + 0.00,
               Orientation == 0 & HorizontalVelocity > 0 & SelfMotion == 4 & Gravity == -9.81 ~ 1 + 0.05 + 0.10 + 0.00,
               Orientation == 0 & HorizontalVelocity > 0 & SelfMotion == 5 & Gravity == -9.81 ~ 1 + 0.05 + 0.00 + 0.00,

               Orientation == 1 & HorizontalVelocity > 0 & SelfMotion == 1 & Gravity ==  9.81 ~ 1 - 0.10 - 0.10 + 0.00,
               Orientation == 1 & HorizontalVelocity > 0 & SelfMotion == 2 & Gravity ==  9.81 ~ 1 - 0.10 - 0.10 + 0.00,
               Orientation == 1 & HorizontalVelocity > 0 & SelfMotion == 3 & Gravity ==  9.81 ~ 1 - 0.10 + 0.10 + 0.00,
               Orientation == 1 & HorizontalVelocity > 0 & SelfMotion == 4 & Gravity ==  9.81 ~ 1 - 0.10 + 0.10 + 0.00,
               Orientation == 1 & HorizontalVelocity > 0 & SelfMotion == 5 & Gravity ==  9.81 ~ 1 - 0.10 + 0.00 + 0.00,
               
               Orientation == 1 & HorizontalVelocity < 0 & SelfMotion == 1 & Gravity ==  9.81 ~ 1 - 0.10 - 0.10 + 0.00,
               Orientation == 1 & HorizontalVelocity < 0 & SelfMotion == 2 & Gravity ==  9.81 ~ 1 - 0.10 + 0.10 + 0.00,
               Orientation == 1 & HorizontalVelocity < 0 & SelfMotion == 3 & Gravity ==  9.81 ~ 1 - 0.10 + 0.10 + 0.00,
               Orientation == 1 & HorizontalVelocity < 0 & SelfMotion == 4 & Gravity ==  9.81 ~ 1 - 0.10 - 0.10 + 0.00,
               Orientation == 1 & HorizontalVelocity < 0 & SelfMotion == 5 & Gravity ==  9.81 ~ 1 - 0.10))
               
               #How did we arrive at these estimates?
               #Upwards gravity should lead to more excentric responses because the vertical velocity component is underestimated, 
               #so the same horizontal velocity should carry the target further (+/-0.05)
               #sidewards gravity should lead to a downwards displacement, independently of initial perpendicular velocity (-0.1)
               
               #selfmotion in the same direction as the perpendicular motion component should lead to an underestimation of the perpendicular motion component
               #and thus to an underestimation of the distance travelled (-0.1 when perpendicular motion and selfmotion go up or to the right; +0.1 when
               #perpendicular motion and selfmotion go down or left)
               #selfmotion opposite to the perpendicular motion should lead to an overestimation of the perpendicular motion component, 
               #and thus to an overestimation of the distance travelled (+0.1 when perpendicular motion go up or to the right; 
               #-0.1 when perpendicular motion goes does or left)
               
               #these selfmotion biases should be reduced by 0.05 for earth gravity.
  
  Dataframe = Dataframe %>%
    group_by(ID) %>%
    mutate(
      ####TIMING
      #add noise to the mean differences in variability established above and scale to occlusion duration
      SD_Time = abs(0.1*OccludedTime*rnorm(1,1,0.1)*SD_Factor_Time),
      #add noise to the mean differences in means and scale to Occlusion Duration
      Mean_Time = OccludedTime*Mean_Factor_Time-OccludedTime + rnorm(1,0,0.03),
      
      ####SPATIAL TASK
      #add noise to the mean differences in variability established above and scale to occlusion duration
      SD_Space = abs(0.1*OccludedDistance*rnorm(1,1,0.1)*SD_Factor_Space),
      #add noise to the mean differences in means and scale to Occlusion Duration
      Mean_Space = OccludedDistance*Mean_Factor_Space-OccludedDistance + rnorm(1,0,0.03))
  
  Dataframe = Dataframe %>% 
    mutate(response_Time = rnorm(length(Mean_Time),Mean_Time,SD_Time))
  
  Dataframe = Dataframe %>% 
    mutate(response_Space = rnorm(length(Mean_Space),Mean_Space,SD_Space))
  
  Dataframe = Dataframe %>%
    mutate(Ratio_Time = (response_Time+OccludedTime)/OccludedTime,
           Ratio_Space = response_Space/abs(OccludedDistance))
  
  Dataframe = Dataframe %>%
    mutate(
      GravityAsFactor = 
        case_when(
          Gravity == 9.81 & Orientation == 0 ~ "Gravity",
          Gravity == -9.81  & Orientation == 0 ~ "InvertedGravity",
          TRUE ~ "SidewardsGravity"
        )
    )
  
  Dataframe = Dataframe %>% 
    mutate(SelfMotionImpactsSpaceAccuracy =
             case_when(SelfMotion == 5 ~ "1Static",
                       
                       SelfMotion == 1 & HorizontalVelocity > 0 & GravityAsFactor == "Gravity" ~ "2OppositeToGravity", #undershoot
                       SelfMotion == 1 & HorizontalVelocity > 0 & GravityAsFactor == "InvertedGravity" ~ "3SameAsGravity", #overshoot
                       SelfMotion == 1 & HorizontalVelocity > 0 & GravityAsFactor == "SidewardsGravity" ~ "4SameAsPerpVelocity", #undershoot
                       
                       SelfMotion == 2 & HorizontalVelocity > 0 & GravityAsFactor == "Gravity" ~ "4SameAsPerpVelocity", #undershoot
                       SelfMotion == 2 & HorizontalVelocity > 0 & GravityAsFactor == "InvertedGravity" ~ "4SameAsPerpVelocity", #undershoot
                       SelfMotion == 2 & HorizontalVelocity > 0 & GravityAsFactor == "SidewardsGravity" ~ "2OppositeToGravity",#undershoot
                       
                       SelfMotion == 3 & HorizontalVelocity > 0 & GravityAsFactor == "Gravity" ~ "3SameAsGravity",#overshoot
                       SelfMotion == 3 & HorizontalVelocity > 0 & GravityAsFactor == "InvertedGravity" ~ "2OppositeToGravity",#undershoot
                       SelfMotion == 3 & HorizontalVelocity > 0 & GravityAsFactor == "SidewardsGravity" ~ "5OppositeToPerpVelocity",#overshoot
                       
                       SelfMotion == 4 & HorizontalVelocity > 0 & GravityAsFactor == "Gravity" ~ "5OppositeToPerpVelocity",#overshoot
                       SelfMotion == 4 & HorizontalVelocity > 0 & GravityAsFactor == "InvertedGravity" ~ "5OppositeToPerpVelocity",#overshoot
                       SelfMotion == 4 & HorizontalVelocity > 0 & GravityAsFactor == "SidewardsGravity" ~ "3SameAsGravity",#undershoot
                       
                       SelfMotion == 1 & HorizontalVelocity < 0 & GravityAsFactor == "Gravity" ~ "2OppositeToGravity",
                       SelfMotion == 1 & HorizontalVelocity < 0 & GravityAsFactor == "InvertedGravity" ~ "3SameAsGravity",
                       SelfMotion == 1 & HorizontalVelocity < 0 & GravityAsFactor == "SidewardsGravity" ~ "5OppositeToPerpVelocity",
                       
                       SelfMotion == 2 & HorizontalVelocity < 0 & GravityAsFactor == "Gravity" ~ "5OppositeToPerpVelocity",
                       SelfMotion == 2 & HorizontalVelocity < 0 & GravityAsFactor == "InvertedGravity" ~ "5OppositeToPerpVelocity",
                       SelfMotion == 2 & HorizontalVelocity < 0 & GravityAsFactor == "SidewardsGravity" ~ "2OppositeToGravity",
                       
                       SelfMotion == 3 & HorizontalVelocity < 0 & GravityAsFactor == "Gravity" ~ "3SameAsGravity",
                       SelfMotion == 3 & HorizontalVelocity < 0 & GravityAsFactor == "InvertedGravity" ~ "2OppositeToGravity",
                       SelfMotion == 3 & HorizontalVelocity < 0 & GravityAsFactor == "SidewardsGravity" ~ "4SameAsPerpVelocity",
                       
                       SelfMotion == 4 & HorizontalVelocity < 0 & GravityAsFactor == "Gravity" ~ "4SameAsPerpVelocity",
                       SelfMotion == 4 & HorizontalVelocity < 0 & GravityAsFactor == "InvertedGravity" ~ "4SameAsPerpVelocity",
                       SelfMotion == 4 & HorizontalVelocity < 0 & GravityAsFactor == "SidewardsGravity" ~ "3SameAsGravity",
             ),
           SelfMotionForSpace = case_when(
             SelfMotionImpactsSpaceAccuracy %in% c("3SameAsGravity","5OppositeToPerpVelocity") & HorizontalVelocity > 0 ~ "2OvershootPositive",
             SelfMotionImpactsSpaceAccuracy %in% c("3SameAsGravity","5OppositeToPerpVelocity") & HorizontalVelocity < 0 ~ "3OvershootNegative",
             SelfMotionImpactsSpaceAccuracy %in% c("2OppositeToGravity","4SameAsPerpVelocity") & HorizontalVelocity > 0 ~ "4UndershootPositive",
             SelfMotionImpactsSpaceAccuracy %in% c("2OppositeToGravity","4SameAsPerpVelocity") & HorizontalVelocity < 0 ~ "5UndershootNegative",
             SelfMotionImpactsSpaceAccuracy == "1Static" ~ "1Static"),
           ObjectMotionFactor =
             case_when(GravityAsFactor == "Gravity" & HorizontalVelocity > 0 ~ "GravityRight",
                       GravityAsFactor == "Gravity" & HorizontalVelocity < 0 ~ "GravityLeft",
                       GravityAsFactor == "InvertedGravity" & HorizontalVelocity > 0 ~ "InvertedGravityRight",
                       GravityAsFactor == "InvertedGravity" & HorizontalVelocity < 0 ~ "InvertedGravityLeft",
                       GravityAsFactor == "SidewardsGravity" & HorizontalVelocity > 0 ~ "SidewardsGravityUp",
                       GravityAsFactor == "SidewardsGravity" & HorizontalVelocity < 0 ~ "SidewardsGravityDown")
        )

  Dataframe = Dataframe %>% 
    mutate(SelfMotionImpactsTimingAccuracy =
             case_when(SelfMotion == 1 & GravityAsFactor == "InvertedGravity" ~ "2SameDirection",
                       SelfMotion == 3 & GravityAsFactor == "Gravity" ~ "2SameDirection",
                       SelfMotion == 1 & GravityAsFactor == "Gravity" ~ "3OppositeDirection",
                       SelfMotion == 3 & GravityAsFactor == "InvertedGravity" ~ "3OppositeDirection",
                       SelfMotion == 2 & GravityAsFactor == "SidewardsGravity" ~ "3OppositeDirection",
                       SelfMotion == 4 & GravityAsFactor == "SidewardsGravity" ~ "2SameDirection",
                       TRUE ~ "1HorizontalOrStatic"
                       )
          )
  Dataframe

}
""";

In [20]:
R"""
AnalysisTimeAccuracy = function(Dataframe){

  mod1 = lmer(Ratio_Time ~ 
                GravityAsFactor*
                SelfMotionImpactsTimingAccuracy +
                (1  | ID),
              data = Dataframe,
              REML = TRUE)
  print(summary(mod1)$coefficients[38:45])
}                   
""";

In [4]:
R"""
AnalysisSpaceAccuracy = function(Dataframe){ #double-checked

  mod1 = lmer(Ratio_Space ~ 
                ObjectMotionFactor*
                SelfMotionForSpace +
                (1  | ID),
              data = Dataframe,
              REML = FALSE)
  print(summary(mod1)$coefficients[74:90])
}                      
""";

In [5]:
R"""
DataframeSDs_Time = function(Dataframe){
  Dataframe = Dataframe %>% 
    mutate(SelfmotionImpactsTimingPrecision =
             case_when(SelfMotion %in% c(1,3) & GravityAsFactor %in% c("Gravity", "InvertedGravity") ~ "2ImpactOfSelfmotion",
                       SelfMotion %in% c(2,4) & GravityAsFactor == c("SidewardsGravity") ~ "2ImpactOfSelfmotion",
                       TRUE ~ "1NoImpactOfSelfmotion"
             )
    )
  
  Dataframe2 = Dataframe %>%
    group_by(SelfMotion,GravityAsFactor,ObjectDirection,ID) %>%
    mutate(SD_Timing = sd(Ratio_Time)) %>%
    slice(1)
  print("DataframeSDs_Time")
  Dataframe2
}                       
""";

In [6]:
R"""
AnalysisTimePrecision_PowerAnalysis = function(DataframeSDs_Time){

  print(DataframeSDs_Time)

  mod1 = lmer(SD_Timing ~ 
                GravityAsFactor*
                SelfmotionImpactsTimingPrecision + 
                (1 | ID),
              data = DataframeSDs_Time,
              REML = TRUE)

  summary(mod1)$coefficient[26:30]
}                      
""";

In [7]:
R"""
DataframeSDs_Space = function(Dataframe){
  Dataframe = Dataframe %>% 
    mutate(SelfMotionImpactsSpacePrecision =
             case_when(SelfMotion == 5 ~ "1NoImpact",
                       TRUE ~ "2Impact")
    )
  
  Dataframe2 = Dataframe %>%
    group_by(SelfMotion,GravityAsFactor,ObjectDirection,ID) %>%
    mutate(SD_Space = sd(Ratio_Space)) %>%
    slice(1)
  print("DataframeSDs_Space")
  Dataframe2
}                     
""";

In [8]:
R"""
AnalysisSpacePrecision_PowerAnalysis = function(DataframeSDs_Space){
print(DataframeSDs_Space)
  mod2 = lmer(SD_Space ~ 
                GravityAsFactor*
                SelfMotionImpactsSpacePrecision + 
                (1 | ID),
              data = DataframeSDs_Space,
              REML = FALSE)
    print("AnalysisSpacePrecision_PowerAnalysis")
  summary(mod2)$coefficient[26:30]
}                        
""";

In [10]:
function PowerAnalysis(n,
        SelfMotion,
        Gravity,
        IntervalVerticalVelocity,
        IntervalHorizontalVelocity,
        reps,
        ObjectDirection,
        Orientation,
        SlantInterval)

    @rput n SelfMotion Gravity IntervalVerticalVelocity IntervalHorizontalVelocity reps ObjectDirection Orientation SlantInterval

    R"""
    ID = paste0("s",1:n)
    Dataframe = SimulateDataframe(n,
            SelfMotion,
            Gravity,
            IntervalVerticalVelocity,
            IntervalHorizontalVelocity,
            reps,
            ObjectDirection,
            Orientation,
            SlantInterval)
    
   # AnalysisTimeAccuracy = AnalysisTimeAccuracy(Dataframe)
   # AnalysisSpaceAccuracy =  AnalysisSpaceAccuracy(Dataframe)
    
   # DataframeSDs_Time = DataframeSDs_Time(Dataframe)
   # DataframeSDs_Space = DataframeSDs_Space(Dataframe)
    
   # AnalysisTimePrecision_PowerAnalysis = AnalysisTimePrecision_PowerAnalysis(DataframeSDs_Time)
   # AnalysisSpacePrecision_PowerAnalysis = AnalysisSpacePrecision_PowerAnalysis(DataframeSDs_Space)
    
    """
   # @rget Dataframe AnalysisTimeAccuracy AnalysisSpaceAccuracy AnalysisTimePrecision_PowerAnalysis AnalysisSpacePrecision_PowerAnalysis
    
    Psychometric[:ObjectDirectionAsFactor] = "placeholder"
    
    for i = 1:length(Psychometric[!,:ObjectDirection])
        Psychometric[i,:ObjectDirectionAsFactor] = string(Psychometric[i,:ObjectDirection])
    end
    
    Psychometric[!,:ObjectDirectionAsFactor] = CategoricalArray(Psychometric[!,:ObjectDirectionAsFactor])
    
    formulaTime = @formula(Answer ~ 1 + ConditionOfInterest + (1 + Difference|ID) + (1 + Difference|StandardValuesAsFactor));
    modelTime = GeneralizedLinearMixedModel(formulaTime, Psychometric, Bernoulli())
    
    formula = @formula(Answer ~ Difference*ConditionOfInterest + (Difference|ID) + (Difference|StandardValuesAsFactor));
    model = GeneralizedLinearMixedModel(formula, Psychometric, Binomial())
    [(coeftable(model)).cols[4][2];(coeftable(model)).cols[4][4]]
    
end

PowerAnalysis (generic function with 1 method)

In [ ]:
nIterations = 1000 #how many data sets do we want to simulate for each power value
Range_Participants = [10,12,14,16,18,20,22,24,26,28,30] #numbers of subjects we want to calculate the power for
SelfMotion = [1:5] #5 selfmotion directions (up, right, down, left, static)
Gravity = [9.81,-9.81]
IntervalVerticalVelocity = [-2*0.375+6.25,2*0.375+6.25]
IntervalHorizontalVelocity = [-2*0.25+3,2*0.25+3]
ObjectDirection = [-1,1]
Range_reps = [10,15,20,25,30] #how many repetitions per condition?
Orientation = [0,1]
SlantInterval = [-8,8]


TotalNumber = length(Range_reps)*length(Range_Participants)
CurrentRunthrough = 0

for reps in Range_reps
    for n in Range_Participants
        for j in 1:1000
            Pvalues = PowerAnalysis(n,
                                      SelfMotion, 
                                      Gravity, 
                                      IntervalVerticalVelocity, 
                                      IntervalHorizontalVelocity, 
                                      reps,
                                      ObjectDirection,
                                      Orientation,
                                      SlantInterval)
            if j = 1
                global DataFrame1 = DataFrame(Pvalues)
            else
                 DataFrame1 = append!(DataFrame1,row)
            end
                
                CurrentRunthrough = CurrentRunthrough + 1
                
                if CurrentRunthrough == 1

                   global PowerfulDataframe = DataFrame(n=n, ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],StandardValue2=StandardValues[2], reps=reps, 
                        PSE_Difference=PSE_Difference, JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard, 
                        SD_ResponseFunction=SD_ResponseFunction, Mean_Variability_Between=Mean_Variability_Between, 
                        SD_Variability_Between=SD_Variability_Between, power_Accuracy = mean(Pvalues_Accuracy .< 0.05),  
                        power_Precision = mean(Pvalues_Precision .< 0.05))    

                else
                    row = DataFrame(n=n, ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],StandardValue2=StandardValues[2], reps=reps, 
                        PSE_Difference=PSE_Difference, JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard, 
                        SD_ResponseFunction=SD_ResponseFunction, Mean_Variability_Between=Mean_Variability_Between, 
                        SD_Variability_Between=SD_Variability_Between, power_Accuracy = mean(Pvalues_Accuracy .< 0.05),  
                        power_Precision = mean(Pvalues_Precision .< 0.05))
                    
                    PowerfulDataframe = append!(PowerfulDataframe,row)
                end
                
                print("Runthrough ", CurrentRunthrough, " out of ", TotalNumber,": ", n, " ", reps, " ", PSE_Difference, " ", JND_Difference, " ", mean(Pvalues_Accuracy .< 0.05), " ", mean(Pvalues_Precision .< 0.05), " ")

            end
            CSV.write(join([reps,"_", PSE_Difference, "_", JND_Difference, ".csv"]),PowerfulDataframe)
        end
    end
end



Runthrough 1 out of 150: 10 50 -0.2 -0.5 0.466 0.758 Runthrough 2 out of 150: 12 50 -0.2 -0.5 0.467 0.7915 Runthrough 3 out of 150: 14 50 -0.2 -0.5 0.47533333333333333 0.824 Runthrough 4 out of 150: 16 50 -0.2 -0.5 0.47925 0.8505 Runthrough 5 out of 150: 18 50 -0.2 -0.5 0.4774 0.869

In [22]:
CSV.write(join(["_","_","lala", ".csv"]),PowerfulDataframe)

"__lala.csv"

In [10]:
join([20, "_", 55,"_", -0.1, "_", -0.5, ".csv"])

"20_55_-0.1_-0.5.csv"

UndefVarError: UndefVarError: tic not defined

(:cols, :colnms, :rownms, :pvalcol, :teststatcol)

In [40]:
n = 20 #numbers of subjects we want to calculate the power for
SelfMotion = [1,2,3,4,5] #5 selfmotion directions (up, right, down, left, static)
Gravity = [9.81,-9.81]
IntervalVerticalVelocity = [-2*0.375+6.25,2*0.375+6.25]
IntervalHorizontalVelocity = [-2*0.25+3,2*0.25+3]
ObjectDirection = [-1,1]
reps = 55 #how many repetitions per condition?
SlantInterval = [-8,8]
Orientation = [0,1]
n
@rput n SelfMotion Gravity IntervalVerticalVelocity IntervalHorizontalVelocity reps ObjectDirection Orientation SlantInterval

R"""
Dataframe = SimulateDataframe(n,
            SelfMotion,
            Gravity,
            IntervalVerticalVelocity,
            IntervalHorizontalVelocity,
            reps,
            ObjectDirection,
            Orientation,
            SlantInterval)
    
    AnalysisTimeAccuracy_model = AnalysisTimeAccuracy(Dataframe)
    AnalysisSpaceAccuracy_model =  AnalysisSpaceAccuracy(Dataframe)
    
    DataframeSDs_Time_model = DataframeSDs_Time(Dataframe)
    DataframeSDs_Space_model = DataframeSDs_Space(Dataframe)

    AnalysisTimePrecision_PowerAnalysis_model = AnalysisTimePrecision_PowerAnalysis(DataframeSDs_Time_model)
    AnalysisSpacePrecision_PowerAnalysis_model = AnalysisSpacePrecision_PowerAnalysis(DataframeSDs_Space_model)
    
    """
@rget Dataframe AnalysisTimeAccuracy_model AnalysisSpaceAccuracy_model AnalysisTimePrecision_PowerAnalysis_model AnalysisSpacePrecision_PowerAnalysis_model
@rget DataframeSDs_Time_model DataframeSDs_Space_model
ModelTimeAccuracyJulia = fit(MixedModel,@formula(Ratio_Time ~ GravityAsFactor*SelfMotionImpactsTimingAccuracy + (1|ID)),Dataframe)

[1]  0.000000e+00 7.664722e-243  2.151550e-44  4.654279e-38  2.771267e-16
[6]  5.589568e-21  6.279855e-22  1.986718e-16
 [1]  9.923185e-01  2.423076e-19  5.867747e-21  1.678156e-81  8.991861e-78
 [6] 5.755754e-103  1.146912e-26  2.031249e-91  3.259986e-27  6.187763e-12
[11]  3.793772e-01  1.686609e-14  3.398670e-11  1.131007e-11  6.434995e-01
[16]  6.989669e-11  2.116378e-14
[1] "DataframeSDs_Time"
[1] "DataframeSDs_Space"
# A tibble: 600 x 40
# Groups:   SelfMotion, GravityAsFactor, ObjectDirection, ID [600]
   ID    SelfMotion Gravity  reps ObjectDirection Orientation VerticalVelocity
   <fct>      <int>   <dbl> <int>           <int>       <int>            <dbl>
 1 s1             1    9.81     1              -1           0             6.09
 2 s2             1    9.81     1              -1           0             6.29
 3 s3             1    9.81     1              -1           0             5.77
 4 s4             1    9.81     1              -1           0             5.86
 5 s5      

┌ Warning: RCall.jl: fixed-effect model matrix is rank deficient so dropping 12 columns / coefficients
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113



# A tibble: 600 x 39
# Groups:   SelfMotion, GravityAsFactor, ObjectDirection, ID [600]
   ID    SelfMotion Gravity  reps ObjectDirection Orientation VerticalVelocity
   <fct>      <int>   <dbl> <int>           <int>       <int>            <dbl>
 1 s1             1    9.81     1              -1           0             6.09
 2 s2             1    9.81     1              -1           0             6.29
 3 s3             1    9.81     1              -1           0             5.77
 4 s4             1    9.81     1              -1           0             5.86
 5 s5             1    9.81     1              -1           0             5.80
 6 s6             1    9.81     1              -1           0             5.83
 7 s7             1    9.81     1              -1           0             5.59
 8 s8             1    9.81     1              -1           0             6.99
 9 s9             1    9.81     1              -1           0             6.31
10 s10            1    9.81     1         

Linear mixed model fit by maximum likelihood
 Ratio_Time ~ 1 + GravityAsFactor + SelfMotionImpactsTimingAccuracy + GravityAsFactor & SelfMotionImpactsTimingAccuracy + (1 | ID)
     logLik        -2 logLik          AIC             BIC       
  2.19089611×10⁴ -4.38179222×10⁴ -4.37959222×10⁴ -4.37034753×10⁴

Variance components:
            Column     Variance   Std.Dev.  
ID       (Intercept)  0.004442967 0.06665559
Residual              0.015461095 0.12434265
 Number of obs: 33000; levels of grouping factors: 20

  Fixed-effects parameters:
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                           Estimate   Std.Error    z value  P(>|z|)
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
(Intercept)                                               

In [41]:
AnalysisTimePrecision_PowerAnalysis_model

5-element Array{Float64,1}:
 8.378562440667714e-47
 4.879185672632236e-38
 5.959884194319933e-5
 0.017450128937099633
 0.0003362509281948972

In [15]:
AnalysisTimeAccuracy

8-element Array{Float64,1}:
 1.9773668366295687e-11
 2.939534777981976e-6
 0.6727512835556073
 0.7734905536262147
 0.45588441099453214
 0.2144959356867396
 0.11625409881230722
 0.031716528487926665

In [42]:
AnalysisSpacePrecision_PowerAnalysis_model

5-element Array{Float64,1}:
 4.586641143549185e-9
 2.476694240220562e-10
 4.079310319355601e-6
 0.0003284310854658815
 0.00011743970812414413

In [97]:
datf = joinpath(dirname(pathof(MixedModels)),"..","test","dat.rda")
dat = Dict(Symbol(k)=>v for (k,v) in load(datf))
contraform = @formula(use ~ 1+a+abs2(a)+l+urb+(1|d));

@time gm2 = fit!(GeneralizedLinearMixedModel(contraform,
    dat[:Contraception], Bernoulli()), nAGQ=9)

  4.975709 seconds (15.16 M allocations: 628.460 MiB, 2.65% gc time)


Generalized Linear Mixed Model fit by maximum likelihood (nAGQ = 9)
  use ~ 1 + a + :(abs2(a)) + l + urb + (1 | d)
  Distribution: Bernoulli{Float64}
  Link: LogitLink()

  Deviance: 2372.4589

Variance components:
     Column    Variance   Std.Dev. 
d (Intercept)  0.22224878 0.4714327

 Number of obs: 1934; levels of grouping factors: 60

Fixed-effects parameters:
─────────────────────────────────────────────────────────
                Estimate    Std.Error    z value  P(>|z|)
─────────────────────────────────────────────────────────
(Intercept)  -1.03542     0.171935     -6.02217    <1e-8
a             0.00353225  0.00909431    0.388402   0.6977
abs2(a)      -0.00456329  0.000714444  -6.38719    <1e-9
l: 1          0.815128    0.159787      5.10134    <1e-6
l: 2          0.916494    0.182361      5.02571    <1e-6
l: 3+         0.91539     0.183022      5.00152    <1e-6
urb: Y        0.696713    0.118143      5.89719    <1e-8
─────────────────────────────────────────────────────────

In [99]:
dat[:Contraception]

,w,d,use,l,a,urb
,Categorical…,Categorical…,Categorical…,Categorical…,Float64,Categorical…
1,1,1,N,3+,18.44,Y
2,2,1,N,0,-5.5599,Y
3,3,1,N,2,1.44,Y
4,4,1,N,3+,8.44,Y
5,5,1,N,0,-13.559,Y
6,6,1,N,0,-11.56,Y
7,7,1,N,3+,18.44,Y
8,8,1,N,3+,-3.5599,Y
9,9,1,N,1,-5.5599,Y
